# 预备知识

## 安装&导入包

In [4]:
# conda install pytorch torchvision torchaudio -c pytorch

import numpy as np
import torch

## 初始化向量

In [14]:
x = torch.empty(4, 3)
x = torch.randn(4, 3) # randn正态分布、rand均匀分布
x = torch.zeros(4, 3, dtype=torch.long) # 64位整数

x = torch.tensor([5.5, 3]) # 常数向量
x

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])

In [50]:
# torch.XX_like: 生成和x相同size的新向量
y = torch.randn_like(x, dtype=torch.float64)
y

tensor([[ 0.8127, -0.8060]], dtype=torch.float64)

In [53]:
print(x.shape)
print(x.size())

torch.Size([1, 2])
torch.Size([1, 2])


## 运算操作符

In [61]:
# 向量运算
x = torch.ones(3, 2)
y = torch.rand(3, 2)

# x + y
# torch.add(x, y)
# x.add(y)
res = torch.empty_like(x)
torch.add(x, y, out=res)
res

tensor([[1.9874, 1.3534],
        [1.1401, 1.7519],
        [1.4237, 1.3908]])

In [72]:
# pytorch操作inplace版本均有后缀"_" 
x = torch.ones(3, 2)
print('origin x:', x)
x.add_(y)
print('inplace x:', x)

origin x: tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
inplace x: tensor([[1.9874, 1.3534],
        [1.1401, 1.7519],
        [1.4237, 1.3908]])


## 改变形状

In [93]:
# y、z与x同源，一个变化其余跟着变化
x = torch.ones(3, 4)
print(x)
y = x.view(2, 6)
z = x.reshape(2, 6)
y += 1
print(x)
print(z)

y = x.clone() # 取消同源


tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]])
tensor([[2., 2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2., 2.]])


## Tensor和Numpy转换

In [99]:
# 将标量tensor转化为python number
x = torch.randn(1)
x_np = x.item()
print(x)
print(x_np)

tensor([-0.1837])
-0.18372656404972076


In [103]:
# Tensor to Numpy(a, b共享内存)
a = torch.ones(3)
b = a.numpy()
print(a, b)

a += 1
print(a, b)

# Numpy to Tensor(a, b共享内存)
a = np.ones(3)
b = torch.from_numpy(a)
print(a, b)

a += 1
print(a, b)

# 不共享内存的方法：
a = np.ones(3)
b = torch.tensor(a)

tensor([1., 1., 1.]) [1. 1. 1.]
tensor([2., 2., 2.]) [2. 2. 2.]
[1. 1. 1.] tensor([1., 1., 1.], dtype=torch.float64)
[2. 2. 2.] tensor([2., 2., 2.], dtype=torch.float64)


##  自动求梯度

In [125]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()
print(x.grad_fn, y.grad_fn)
print(x.is_leaf, y.is_leaf)
print(out)

# 反向传播，计算梯度
out.backward() # output维度是1，所以backward里默认是Tensor(1)
print(x.grad)


y.backward(torch.ones_like(x)) # y维度不是1，所以需要使用同行向量适配维度
print(x.grad) # 每次backward会叠加，所以需要清空

x.grad.data.zero_()
y.backward(torch.ones_like(x))
print(x.grad)

x.grad.data.zero_()
v = torch.Tensor([[1, 0.1], [0.01, 0.001]])
y.backward(v)
print(x.grad)

None <AddBackward0 object at 0x7fc1fdf79e10>
True False
tensor(27., grad_fn=<MeanBackward0>)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
tensor([[5.5000, 5.5000],
        [5.5000, 5.5000]])
tensor([[1., 1.],
        [1., 1.]])
tensor([[1.0000, 0.1000],
        [0.0100, 0.0010]])


# 深度学习基础 

In [14]:
import torch
# from torch import nn
from collections import OrderedDict

### 搭建网络的四种方式

In [31]:
'''
方式一：直接搭建
'''
class Net1(nn.Module):
    # 初始化，在init里搭建网络
    def __init__(self):
        super(Net1, self).__init__()
        self.conv = nn.Conv2d(3, 32, 3, 1, 1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.dense1 = nn.Linear(32 * 3 * 3, 128)
        self.dense2 = nn.Linear(128, 10)

    # 假设x为input变量，前向计算出需要返回的结果output
    def forward(self, x):
        a1 = self.pool(self.relu(self.conv(x))).reshape(x.size(0), -1)
        a2 = self.dense2(self.relu(self.dense1(a1)))
        return a2

net1 = Net1()
print(net1)

'''
方式二：利用Sequential类
'''
class Net2(nn.Module):
    def __init__(self):
        super(Net2, self).__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, 3, 1, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2))
        self.dense = torch.nn.Sequential(
            torch.nn.Linear(32 * 3 * 3, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 10))

    def forward(self, x):
        conv_out = self.conv(x)
        res = conv_out.reshape(conv_out.size(0), -1)
        out = self.dense(res)
        return out

net2 = Net2()
print(net2)

'''
方式三：利用add_module方法
'''
class Net3(nn.Module):
    def __init__(self):
        super(Net3, self).__init__()
        self.conv = torch.nn.Sequential()
        self.conv.add_module("conv1", torch.nn.Conv2d(3, 32, 3, 1, 1))
        self.conv.add_module("relu1", torch.nn.ReLU())
        self.conv.add_module("pool1", torch.nn.MaxPool2d(2))
        self.dense = torch.nn.Sequential()
        self.dense.add_module('dense1', torch.nn.Linear(32 * 3 * 3, 128))
        self.dense.add_module('relu2', torch.nn.ReLU())
        self.dense.add_module('dense2', torch.nn.Linear(128, 10))
    
    def forward(self, x):
        conv_out = self.conv(x)
        res = conv_out.reshape(conv_out.size(0), -1)
        out = self. dense(res)
        return out

net3 = Net3()
print(net3)


'''
方式三：利用有序字典传入模块
'''
class Net4(nn.Module):
    def __init__(self):
        super(Net4, self).__init__()
        self.conv  = torch.nn.Sequential(
            OrderedDict(
                [
                    ("conv1", torch.nn.Conv2d(3, 32, 3, 1, 1)),
                    ("Relu1", torch.nn.ReLU()),
                    ("pool", torch.nn.MaxPool2d(2))
                ]))
        self.dense  = torch.nn.Sequential(
            OrderedDict(
                [
                    ("dense1", torch.nn.Linear(32 * 3 * 3, 128)),
                    ("Relu2", torch.nn.ReLU()),
                    ("dense2", torch.nn.Linear(128, 10))
                ])) 

    def forward(self, x):
        conv_out = self.conv(x)
        res = conv_out.reshape(conv_out.size(0), -1)
        out = self. dense(res)
        return out

net4 = Net4()
print(net4)

Net1(
  (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dense1): Linear(in_features=288, out_features=128, bias=True)
  (dense2): Linear(in_features=128, out_features=10, bias=True)
)
Net2(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=288, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=10, bias=True)
  )
)
Net3(
  (conv): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (dense1): Linear(in_features=288, out_features=128, bias=True)
    